In [8]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\wjbok\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\wjbok\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\wjbok\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\wjbok\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [4]:
#Record 불러와서 DataFrame으로 변환
df=pd.read_csv('naver_blog_crawl_merged_results_AD.csv')
print('record number',len(df))

record number 5061


In [17]:
#결측치 제거 (내용, 문맥에 관련한 전처리)
context = df[['내용']]
df1= context.dropna()
print('record number',len(df1))

print(df1)

# '내용' 열을 리스트로 변환
context_list = df1['내용'].tolist()


#확인용
#print(context_list[0])

record number 5061
                                                     내용
0     안녕하세요! 오늘은 추운 겨울, 따끈한 곱창전골에 첫 도전해봤어요. 건대에서 유명한...
1     춘천의 석사동 먹자골목에 자리한 이 식당은 찾기 쉽고 주차장도 마련되어 있어 불편함...
2     안녕하세요! 오늘은 제 인생 첫 곱창전골 도전기를 여러분과 공유하고자 해요. 건대에...
3     양양은 현지인들이 추천하는 맛집 투어로 여행을 즐기는 최고의 장소 중 하나입니다.\...
4     안녕하신가요? 오늘은 송파 지역의 숨은 보석, 초이다이닝 송파점에서의 식사 경험을 ...
...                                                 ...
5056  부산 해운대신상카페\n디저트 타르트 맛집\n보노베리 해운대점\n잇님들♡ 안녕하세요~...
5057  50m\n© NAVER Corp.\n별국수 홈플러스 동광주점\n광주광역시 북구 동문...
5058  얼마 전 한 입에 쏙 넣기 좋은 맛있는 눈꽃 치즈살을 섭렵하고 왔어요.\n입이 황홀...
5059  안녕하세요~\n이번 송도 나들이에서 맛나서 또 가게 된\n맛집이 있어요~맛있어서 또...
5060  50m\n© NAVER Corp.\n에이뿔램 군자점\n서울특별시 광진구 능동로36길...

[5061 rows x 1 columns]
안녕하세요! 오늘은 추운 겨울, 따끈한 곱창전골에 첫 도전해봤어요. 건대에서 유명한 77년생곱도리전골식당에서의 경험 소개할게요.
이곳은 매일 11시 30분부터 밤 11시 30분까지 운영하는데, 점심특선도 준비되어 있어 점심시간에 방문하기에도 좋아요.
식당 입구에는 행운을 상징하는 네잎클로버 그림이 그려져 있어, 긍정적인 기대감을 갖게 했어요.
대기 시간에는 이름을 적고 노란색 의자에 앉아서 기다리면 된답니다. 대표 메뉴가 잘 보이는 곳에 전시되어 있어 메뉴 선택에 도움이 됐어요.
입구에 적힌 위로의 말이

In [24]:
#결측치 제거 (제목은 키워드가 많이 들어갈 수 밖에 없다.->임펙트 중요하므로 파악 필요)
title = df[['제목']]
df2= title.dropna()
print('record number',len(df2))

print(df2)

# '제목' 열을 리스트로 변환
title_list = df2['제목'].tolist()

record number 5061
                                      제목
0     건대맛집 곱도리전골 첫 도전 후기 '77년생 곱도리 전골식당'
1             춘천 주문진게먹고회먹고 - 대게와 킹크랩의 맛집
2     건대에서 맛본 곱창전골의 매력 한번 맛보면 헤어나올 수 없어요
3     양양 맛집 투어: 동호해변 주변에 인정받은 대게 맛집 서핑크랩
4        송리단길맛집 초이다이닝 송파점 기대한만큼 너무 좋은...
...                                  ...
5056     부산 해운대신상카페 디저트 타르트 맛집 보노베리 해운대점
5057   동광주 홈플러스 푸드코트 별국수 두암동 맛집 종류별로 맛보기
5058                   입이 황홀했던 안산 소고기 맛집
5059            맛있어서 또 가는 송도 맛집  옥소반 송도점
5060              군자역 근처 맛집 양고기 전문점 에이뿔램

[5061 rows x 1 columns]


In [31]:
import re
from konlpy.tag import Okt

# Okt 형태소 분석기 객체 생성
okt = Okt()

def preprocess_korean_text(text):
    # 텍스트 정제: 특수 문자, 숫자, 영어 등 제거
    text = re.sub(r'[^가-힣\s]', '', text)
    
    # 토큰화 및 품사 태깅
    tokens = okt.pos(text, stem=True)
    
    # 불용어 목록 (예시)
    stop_words = set(['그리고', '그래서', '그러나', '하지만', '또한', '및', '이것', '저것'])
    
    # 불용어 제거
    processed_tokens = [(word, pos) for word, pos in tokens if word not in stop_words and len(word) > 1]
    
    # 결과 확인을 위한 출력
    print("전처리된 토큰 결과:")
    print(processed_tokens)
    
    # 전처리된 텍스트 반환
    processed_text = ' '.join([word for word, pos in processed_tokens])
    return processed_text

# 예시 텍스트(test용)
#text = "이 텍스트는 한국어 전처리를 위한 예시 문장입니다. 다양한 처리 과정을 거칩니다!"

# 전처리된 텍스트를 저장할 리스트
preprocessed_texts = []

# 리스트의 각 요소에 대해 전처리 함수를 적용하여 저장
for text in title_list:
    preprocessed_text = preprocess_korean_text(text)
    preprocessed_texts.append(preprocessed_text)

# 결과 확인
print(preprocessed_texts)



전처리된 토큰 결과:
[('건대', 'Noun'), ('맛집', 'Noun'), ('도리', 'Noun'), ('전골', 'Noun'), ('도전', 'Noun'), ('후기', 'Noun'), ('년생', 'Noun'), ('도리', 'Noun'), ('전골', 'Noun'), ('식당', 'Noun')]
전처리된 토큰 결과:
[('춘천', 'Noun'), ('주문진', 'Noun'), ('먹다', 'Verb'), ('먹다', 'Verb'), ('대게', 'Noun'), ('킹크랩', 'Noun'), ('맛집', 'Noun')]
전처리된 토큰 결과:
[('건대', 'Noun'), ('에서', 'Josa'), ('맛보다', 'Verb'), ('곱창전골', 'Noun'), ('매력', 'Noun'), ('한번', 'Noun'), ('맛보다', 'Verb'), ('헤어', 'Noun'), ('나오다', 'Verb'), ('없다', 'Adjective')]
전처리된 토큰 결과:
[('양양', 'Noun'), ('맛집', 'Noun'), ('투어', 'Noun'), ('동호', 'Noun'), ('해변', 'Noun'), ('주변', 'Noun'), ('인정받다', 'Verb'), ('대게', 'Noun'), ('맛집', 'Noun'), ('서핑', 'Noun'), ('크랩', 'Noun')]
전처리된 토큰 결과:
[('송리', 'Noun'), ('달다', 'Verb'), ('맛집', 'Noun'), ('초이', 'Noun'), ('다이닝', 'Noun'), ('송파', 'Noun'), ('기대하다', 'Adjective'), ('너무', 'Adverb'), ('좋다', 'Adjective')]
전처리된 토큰 결과:
[('인스타', 'Noun'), ('팔로워', 'Noun'), ('늘리다', 'Verb'), ('셀프', 'Noun'), ('도달', 'Noun'), ('성공', 'Noun'), ('후기', 'Noun')]
전처리된 토큰 결과:
[('들다', 'Verb'

전처리된 토큰 결과:
[('제주', 'Noun'), ('서쪽', 'Noun'), ('로컬', 'Noun'), ('맛집', 'Noun'), ('넘사벽', 'Noun'), ('한식', 'Noun'), ('요리', 'Noun'), ('제주', 'Noun'), ('서쪽', 'Noun'), ('맛집', 'Noun')]
전처리된 토큰 결과:
[('만원', 'Noun'), ('감성', 'Noun'), ('숙소', 'Noun'), ('찾다', 'Verb'), ('발견', 'Noun'), ('그림', 'Noun'), ('같다', 'Adjective')]
전처리된 토큰 결과:
[('제주도', 'Noun'), ('애월', 'Noun'), ('돼지', 'Noun'), ('맛집', 'Noun'), ('감탄', 'Noun'), ('하다', 'Verb'), ('제주도', 'Noun'), ('돼지', 'Noun'), ('거리', 'Noun'), ('맛집', 'Noun')]
전처리된 토큰 결과:
[('성산', 'Noun'), ('가성', 'Noun'), ('맛집', 'Noun'), ('어우름', 'Noun'), ('오감', 'Noun'), ('만족하다', 'Adjective'), ('제주', 'Noun'), ('성산', 'Noun'), ('돼지', 'Noun'), ('맛집', 'Noun')]
전처리된 토큰 결과:
[('제주도', 'Noun'), ('협재', 'Noun'), ('맛집', 'Noun'), ('전통', 'Noun'), ('있다', 'Adjective'), ('식당', 'Noun'), ('애월', 'Noun'), ('협재', 'Noun'), ('맛집', 'Noun'), ('해녀', 'Noun'), ('세자매', 'Noun')]
전처리된 토큰 결과:
[('쫀득살', 'Noun'), ('유명하다', 'Adjective'), ('프랜차이즈', 'Noun'), ('맛집', 'Noun'), ('조부', 'Noun'), ('안집', 'Noun'), ('신도림', 'Noun'), ('본점', 

전처리된 토큰 결과:
[('시장', 'Noun'), ('맛집', 'Noun'), ('제주', 'Noun'), ('에도', 'Josa'), ('퀄리티', 'Noun'), ('대게', 'Noun'), ('랍스타', 'Noun'), ('먹다', 'Verb')]
전처리된 토큰 결과:
[('미금역', 'Noun'), ('초밥', 'Noun'), ('사시미', 'Noun'), ('맛집', 'Noun'), ('초밥', 'Noun')]
전처리된 토큰 결과:
[('광주', 'Noun'), ('맛집', 'Noun'), ('베트남', 'Noun'), ('광주', 'Noun'), ('만나다', 'Verb'), ('하노이', 'Noun'), ('맛집', 'Noun'), ('후기', 'Noun')]
전처리된 토큰 결과:
[('건대', 'Noun'), ('맛집', 'Noun'), ('도리', 'Noun'), ('성하다', 'Adjective'), ('후기', 'Noun'), ('년생', 'Noun'), ('도리', 'Noun'), ('전골', 'Noun'), ('식당', 'Noun')]
전처리된 토큰 결과:
[('어메', 'Exclamation'), ('가득', 'Noun'), ('게장', 'Noun'), ('아니다', 'Adjective')]
전처리된 토큰 결과:
[('인스타그램', 'Noun'), ('감성', 'Noun'), ('사진', 'Noun'), ('맛집', 'Noun'), ('라운지', 'Noun')]
전처리된 토큰 결과:
[('여수', 'Noun'), ('낭만', 'Noun'), ('별식', 'Noun'), ('감동', 'Noun'), ('마리', 'Noun'), ('토끼', 'Noun'), ('잡다', 'Verb')]
전처리된 토큰 결과:
[('여수', 'Noun'), ('맛집', 'Noun'), ('낭만', 'Noun'), ('별식', 'Noun'), ('게장', 'Noun'), ('놓치다', 'Verb'), ('돼다', 'Verb'), ('여수', 'Noun'), (

전처리된 토큰 결과:
[('제주', 'Noun'), ('공항', 'Noun'), ('근처', 'Noun'), ('맛집', 'Noun'), ('한라', 'Noun'), ('오름', 'Noun'), ('게우다', 'Verb'), ('궁금하다', 'Adjective')]
전처리된 토큰 결과:
[('빨다', 'Verb'), ('빨다', 'Verb'), ('퍼블릭', 'Noun'), ('가산', 'Noun'), ('가산', 'Noun'), ('디지털', 'Noun'), ('단지', 'Noun'), ('맛집', 'Noun'), ('닭갈비', 'Noun'), ('전문점', 'Noun')]
전처리된 토큰 결과:
[('제주', 'Noun'), ('함덕', 'Noun'), ('맛집', 'Noun'), ('곱들락', 'Noun'), ('함덕', 'Noun'), ('돼지', 'Noun'), ('함덕', 'Noun'), ('해수욕장', 'Noun'), ('맛집', 'Noun'), ('곱들락', 'Noun')]
전처리된 토큰 결과:
[('용인', 'Noun'), ('맛집', 'Noun'), ('한식', 'Noun'), ('그렇다', 'Adjective'), ('용인역', 'Noun'), ('직영', 'Noun')]
전처리된 토큰 결과:
[('청라', 'Noun'), ('소고기', 'Noun'), ('레스토랑', 'Noun'), ('맛집', 'Noun'), ('숙성', 'Noun'), ('명작', 'Noun')]
전처리된 토큰 결과:
[('광안', 'Noun'), ('유럽', 'Noun'), ('카페', 'Noun'), ('디저트', 'Noun'), ('맛집', 'Noun'), ('광안', 'Noun'), ('라크', 'Noun'), ('초콜릿', 'Noun')]
전처리된 토큰 결과:
[('전라남도', 'Noun'), ('맛집', 'Noun'), ('여수', 'Noun'), ('게장', 'Noun'), ('맛집', 'Noun'), ('여수', 'Noun'), ('게장', 'Noun'),

전처리된 토큰 결과:
[('김해진', 'Noun'), ('맛집', 'Noun'), ('한우', 'Noun'), ('곱창', 'Noun'), ('순대', 'Noun'), ('전골', 'Noun'), ('먹다', 'Verb'), ('그렇다', 'Adjective'), ('바로', 'Noun')]
전처리된 토큰 결과:
[('서핑', 'Noun'), ('대게', 'Noun'), ('낮술', 'Noun'), ('진리', 'Noun'), ('양양', 'Noun'), ('맛집', 'Noun'), ('서핑', 'Noun'), ('크랩', 'Noun')]
전처리된 토큰 결과:
[('서울숲', 'Noun'), ('소개팅', 'Noun'), ('파스타', 'Noun'), ('서울숲', 'Noun'), ('화이트', 'Noun'), ('파스타', 'Noun'), ('맛집', 'Noun')]
전처리된 토큰 결과:
[('첨단', 'Noun'), ('통치', 'Noun'), ('돈까스', 'Noun'), ('맛집', 'Noun'), ('첨단', 'Noun'), ('필링', 'Noun'), ('돈까스', 'Noun')]
전처리된 토큰 결과:
[('국산', 'Noun'), ('취급', 'Noun'), ('초당', 'Noun'), ('순두부', 'Noun'), ('떡갈비', 'Noun'), ('초당', 'Noun'), ('맛집', 'Noun')]
전처리된 토큰 결과:
[('잠실', 'Noun'), ('새다', 'Verb'), ('맛집', 'Noun'), ('요리', 'Noun'), ('하다', 'Verb'), ('남자', 'Noun'), ('화덕', 'Noun'), ('피자', 'Noun'), ('맛집', 'Noun')]
전처리된 토큰 결과:
[('면식', 'Noun'), ('아주대', 'Noun'), ('아주대', 'Noun'), ('맛집', 'Noun')]
전처리된 토큰 결과:
[('소정', 'Noun'), ('성신여대', 'Noun'), ('성신여대', 'Noun'), ('맛집', 'N

...

전처리된 토큰 결과:
[('최고급', 'Noun'), ('이던', 'Josa'), ('제주', 'Noun'), ('서귀포시', 'Noun'), ('맛집', 'Noun'), ('블랙', 'Noun')]
전처리된 토큰 결과:
[('대전', 'Noun'), ('둔산동', 'Noun'), ('맛집', 'Noun'), ('온유', 'Noun'), ('매운탕', 'Noun'), ('주차', 'Noun'), ('웨이', 'Noun')]
전처리된 토큰 결과:
[('광화문', 'Noun'), ('타워', 'Noun'), ('점심', 'Noun'), ('맛집', 'Noun'), ('한우', 'Noun'), ('마카', 'Noun'), ('식당', 'Noun')]
전처리된 토큰 결과:
[('동해', 'Noun'), ('일식', 'Noun'), ('여의도', 'Noun'), ('회식', 'Noun'), ('장소', 'Noun'), ('국회의사당', 'Noun'), ('맛집', 'Noun'), ('일식', 'Noun'), ('코스', 'Noun'), ('요리', 'Noun')]
전처리된 토큰 결과:
[('광안', 'Noun'), ('가보다', 'Verb'), ('가족', 'Noun'), ('외식', 'Noun'), ('메뉴', 'Noun'), ('샤브샤브', 'Noun'), ('맛집', 'Noun'), ('우봉샤브', 'Noun')]
전처리된 토큰 결과:
[('하남', 'Noun'), ('맛집', 'Noun'), ('퀄리티', 'Noun'), ('좋다', 'Adjective'), ('로스트', 'Noun'), ('피그', 'Noun'), ('미사', 'Noun')]
전처리된 토큰 결과:
[('왕십리', 'Noun'), ('모임', 'Noun'), ('하다', 'Verb'), ('좋다', 'Adjective'), ('갈비', 'Noun'), ('맛집', 'Noun'), ('기와', 'Noun'), ('타운', 'Noun')]
전처리된 토큰 결과:
[('해운대', 'Noun'), ('오

전처리된 토큰 결과:
[('만족', 'Noun'), ('영덕', 'Noun'), ('대게', 'Noun'), ('맛집', 'Noun'), ('추천', 'Noun'), ('사계절', 'Noun'), ('대게', 'Noun')]
전처리된 토큰 결과:
[('광화문', 'Noun'), ('교보', 'Noun'), ('문고', 'Noun'), ('맛집', 'Noun'), ('몰다', 'Verb'), ('신상', 'Noun'), ('곱다', 'Adjective'), ('광화문역', 'Noun'), ('타워', 'Noun'), ('맛집', 'Noun')]
전처리된 토큰 결과:
[('부산', 'Noun'), ('연제구', 'Noun'), ('부산', 'Noun'), ('시청', 'Noun'), ('맛집', 'Noun'), ('골다', 'Verb')]
전처리된 토큰 결과:
[('부산', 'Noun'), ('맛집', 'Noun'), ('에서', 'Josa'), ('스테이크', 'Noun'), ('먹다', 'Verb')]
전처리된 토큰 결과:
[('광교', 'Noun'), ('앨리', 'Noun'), ('웨이', 'Noun'), ('맛집', 'Noun'), ('가족', 'Noun'), ('모임', 'Noun'), ('으로', 'Josa'), ('좋다', 'Adjective'), ('도쿄', 'Noun'), ('등심', 'Noun'), ('광교', 'Noun'), ('에서', 'Josa')]
전처리된 토큰 결과:
[('매일', 'Noun'), ('가다', 'Verb'), ('안산', 'Noun'), ('고잔동', 'Noun'), ('맛집', 'Noun')]
전처리된 토큰 결과:
[('수원', 'Noun'), ('광교', 'Noun'), ('앨리', 'Noun'), ('웨이', 'Noun'), ('맛집', 'Noun'), ('영통', 'Noun'), ('백합', 'Noun'), ('칼국수', 'Noun'), ('후기', 'Noun'), ('오픈', 'Noun')]
전처리된 토큰 결과

전처리된 토큰 결과:
[('김포', 'Noun'), ('구래동', 'Noun'), ('맛집', 'Noun'), ('샤브샤브', 'Noun'), ('무한리필', 'Noun'), ('소담', 'Noun'), ('점심', 'Noun'), ('추천', 'Noun')]
전처리된 토큰 결과:
[('감칠', 'Noun'), ('좋다', 'Adjective'), ('제주', 'Noun'), ('성산일출봉', 'Noun'), ('맛집', 'Noun'), ('해왓', 'Noun')]
전처리된 토큰 결과:
[('대입', 'Noun'), ('구역', 'Noun'), ('맛집', 'Noun'), ('김통년', 'Noun'), ('운영', 'Noun'), ('중인', 'Noun'), ('숙성', 'Noun'), ('고기', 'Noun')]
전처리된 토큰 결과:
[('광안', 'Noun'), ('맛집', 'Noun'), ('부산', 'Noun'), ('여행', 'Noun'), ('찾다', 'Verb'), ('장덕', 'Noun'), ('풍천', 'Noun'), ('장어', 'Noun')]
전처리된 토큰 결과:
[('동명동', 'Noun'), ('일식', 'Noun'), ('맛집', 'Noun'), ('연어쁘', 'Noun'), ('광주', 'Noun'), ('연어', 'Noun'), ('초밥', 'Noun'), ('추천', 'Noun')]
전처리된 토큰 결과:
[('얼큰하다', 'Adjective'), ('안양', 'Noun'), ('칼국수', 'Noun'), ('맛집', 'Noun'), ('평촌역', 'Noun'), ('느리다', 'Adjective'), ('밥상', 'Noun')]
전처리된 토큰 결과:
[('국회의사당역', 'Noun'), ('맛집', 'Noun'), ('마음껏', 'Noun'), ('즐기다', 'Verb'), ('보다', 'Verb'), ('여의도', 'Noun'), ('회전초밥', 'Noun'), ('뷔페', 'Noun')]
전처리된 토큰 결과:
[('광화문', 

전처리된 토큰 결과:
[('여의도', 'Noun'), ('맛집', 'Noun'), ('국회의사당역', 'Noun'), ('회식', 'Noun'), ('장소', 'Noun'), ('찾다', 'Verb'), ('동해', 'Noun'), ('일식', 'Noun'), ('추천', 'Noun')]
전처리된 토큰 결과:
[('코엑스', 'Noun'), ('근처', 'Noun'), ('맛집', 'Noun'), ('도우', 'Noun'), ('에서', 'Josa'), ('삼성역', 'Noun'), ('샤브샤브', 'Noun')]
전처리된 토큰 결과:
[('프라하', 'Noun'), ('숙소', 'Noun'), ('추천', 'Noun'), ('위치', 'Noun'), ('가격', 'Noun'), ('조식', 'Noun'), ('맛집', 'Noun'), ('할인', 'Noun'), ('코드', 'Noun'), ('꿀팁', 'Noun')]
전처리된 토큰 결과:
[('부천역', 'Noun'), ('훠궈', 'Noun'), ('추천', 'Noun'), ('천지', 'Noun'), ('훠궈', 'Noun'), ('샤브샤브', 'Noun'), ('성비', 'Noun'), ('무한리필', 'Noun'), ('맛집', 'Noun')]
전처리된 토큰 결과:
[('서울역', 'Noun'), ('근처', 'Noun'), ('맛집', 'Noun'), ('동화', 'Noun'), ('고옥', 'Noun'), ('서울', 'Noun'), ('정갈하다', 'Adjective'), ('한식당', 'Noun'), ('추천', 'Noun'), ('주차', 'Noun')]
전처리된 토큰 결과:
[('가족', 'Noun'), ('애인', 'Noun'), ('서울', 'Noun'), ('사람', 'Noun'), ('데려가다', 'Verb'), ('있다', 'Adjective'), ('부산', 'Noun'), ('맛집', 'Noun'), ('웨이', 'Noun'), ('주차', 'Noun'), ('정보', 'Nou

전처리된 토큰 결과:
[('광안', 'Noun'), ('리피', 'Noun'), ('도우', 'Noun'), ('개다', 'Verb'), ('광안', 'Noun'), ('피맥', 'Noun'), ('신상', 'Noun'), ('맛집', 'Noun')]
전처리된 토큰 결과:
[('부드럽다', 'Adjective'), ('등심', 'Noun'), ('송리', 'Noun'), ('달다', 'Verb'), ('맛집', 'Noun')]
전처리된 토큰 결과:
[('영덕', 'Noun'), ('까지', 'Josa'), ('갈다', 'Verb'), ('필요', 'Noun'), ('없다', 'Adjective'), ('일산', 'Noun'), ('대게', 'Noun'), ('맛집', 'Noun'), ('대게', 'Noun'), ('일산', 'Noun'), ('백석', 'Noun')]
전처리된 토큰 결과:
[('내공', 'Noun'), ('있다', 'Adjective'), ('써다', 'Verb'), ('밸리', 'Noun'), ('맛집', 'Noun')]
전처리된 토큰 결과:
[('청라', 'Noun'), ('닭갈비', 'Noun'), ('맛집', 'Noun'), ('아랜', 'Noun'), ('역물', 'Noun'), ('닭갈비', 'Noun'), ('구성원', 'Noun'), ('닭갈비', 'Noun')]
전처리된 토큰 결과:
[('서면', 'Noun'), ('햄버거', 'Noun'), ('혼밥', 'Noun'), ('맛집', 'Noun'), ('칙바이칙', 'Noun'), ('서면', 'Noun')]
전처리된 토큰 결과:
[('김해', 'Noun'), ('삼계동', 'Noun'), ('맛집', 'Noun'), ('김해', 'Noun'), ('삼계', 'Noun'), ('가족', 'Noun'), ('외식', 'Noun'), ('장소', 'Noun'), ('추천', 'Noun')]
전처리된 토큰 결과:
[('기법', 'Noun'), ('으로', 'Josa'), ('풍부하다'

In [36]:
def detokenize_text(tokens):
    # 토큰 사이에 공백을 추가하여 문장으로 결합
    return ''.join(tokens)

# 전처리된 토큰에 역토큰화 함수를 적용하여 문서 형식으로 반환
processed_documents = pd.Series(preprocessed_texts).apply(lambda x: detokenize_text(x))

# 결과 확인
print(processed_documents)

0            건대 맛집 도리 전골 도전 후기 년생 도리 전골 식당
1                   춘천 주문진 먹다 먹다 대게 킹크랩 맛집
2       건대 에서 맛보다 곱창전골 매력 한번 맛보다 헤어 나오다 없다
3       양양 맛집 투어 동호 해변 주변 인정받다 대게 맛집 서핑 크랩
4            송리 달다 맛집 초이 다이닝 송파 기대하다 너무 좋다
                       ...                
5056     부산 해운대 신상 카페 디저트 타르트 맛집 보노 베리 해운대
5057          광주 홈플러스 푸드코트 국수 맛집 종류 별로 맛보기
5058                        황홀하다 안산 소고기 맛집
5059                    맛있다 가다 송도 맛집 옥소 송도
5060               군자역 근처 맛집 양고기 전문점 에이 뿔램
Length: 5061, dtype: object


In [55]:
import torch
from transformers import BertTokenizer, BertModel
from tqdm import tqdm

# KoBERT 모델과 토크나이저 불러오기
tokenizer = BertTokenizer.from_pretrained("monologg/kobert")
model = BertModel.from_pretrained("monologg/kobert")

# KoBERT를 사용하여 문서 임베딩 함수 정의
def embed_documents(documents):
    embeddings = []
    # 각 문서에 대해 임베딩 수행
    for document in tqdm(documents):
        # 문서를 토큰화하여 KoBERT의 입력 형식으로 변환
        inputs = tokenizer(document, return_tensors="pt", padding=True, truncation=True)
        # KoBERT 모델에 입력을 전달하여 임베딩 추출
        with torch.no_grad():
            outputs = model(**inputs)
        # 임베딩된 결과 추출 (마지막 레이어의 토큰 임베딩 평균)
        token_embeddings = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
        embeddings.append(token_embeddings)
    return embeddings

# 처리된 텍스트 데이터를 KoBERT를 사용하여 임베딩
embedded_documents = embed_documents(processed_documents)

# 임베딩된 결과 출력
print(embedded_documents[:10])


100%|██████████████████████████████████████████████████████████████████████████████| 5061/5061 [05:14<00:00, 16.08it/s]

[array([ 2.42392495e-01, -5.72872460e-02, -3.02954037e-02, -5.98391867e+00,
        1.72833055e-02,  3.67467925e-02, -8.57245997e-02,  8.57493654e-02,
       -8.41939673e-02, -2.84869403e-01, -2.86902457e-01, -3.73093039e-01,
       -1.33089200e-02, -7.02475086e-02,  5.58979571e-01, -4.68160957e-02,
       -5.94172021e-03, -8.22453052e-02, -3.97223756e-02, -4.05453652e-01,
       -2.87165135e-01,  3.28466743e-02, -1.60411373e-01,  2.08107848e-02,
        2.17665527e-02,  4.62481985e-03,  3.44379582e-02, -1.63861945e-01,
       -2.17355192e-02,  4.69018668e-01, -8.31372812e-02, -1.58005312e-01,
        1.34221362e-02, -1.97444752e-01,  1.73792616e-01, -1.73291862e-01,
        2.57048845e-01,  8.49421263e-01,  3.22533667e-01, -3.04742008e-01,
        4.16597009e-01,  2.51609206e-01,  3.53515357e-01,  4.87712882e-02,
        5.91729023e-02, -1.41199902e-01, -5.11304103e-02,  1.06586494e-01,
        2.50849694e-01,  3.28923196e-01,  5.45158744e-01,  4.55972552e-01,
       -3.71904410e-02, 

In [56]:
from gensim.models import FastText
from gensim.utils import simple_preprocess

# 문서를 토큰화하여 FastText 모델에 학습시키기 위한 형태로 변환
tokenized_documents = [simple_preprocess(document, min_len=1, max_len=100) for document in processed_documents]

# FastText 모델 학습
fasttext_model = FastText(sentences=tokenized_documents, vector_size=100, window=5, min_count=1, workers=4, sg=1)

# 각 문서에 대한 임베딩 추출
embedded_documents_fasttext = [fasttext_model.wv[document] for document in tokenized_documents]

# 임베딩된 결과 출력
print(embedded_documents_fasttext[:1])


[array([[ 1.60739526e-01,  1.59757808e-01, -1.00897804e-01,
        -1.05403475e-01,  1.55393556e-01, -1.00459624e-02,
         3.41285795e-01, -1.29083946e-01, -2.97542870e-01,
        -4.49114963e-02, -5.28071634e-02, -1.61449574e-02,
        -2.29242176e-01, -2.82071263e-01, -7.73823112e-02,
        -1.23226374e-01, -1.03376411e-01,  1.21424824e-01,
         7.45501593e-02, -2.44274259e-01,  1.68425426e-01,
         1.63775086e-02, -1.71157956e-01, -3.22062552e-01,
        -3.42111617e-01, -2.25760907e-01, -1.43079713e-01,
         1.93185687e-01, -8.34928602e-02, -1.06905930e-01,
        -1.24095269e-01,  6.31425232e-02, -2.00559329e-02,
         2.02273726e-01,  4.63231746e-03,  8.19988251e-02,
         3.24836284e-01, -3.31949517e-02, -1.32967919e-01,
        -6.88843876e-02, -2.82683611e-01, -8.51725861e-02,
         1.77411348e-01,  8.26805085e-02,  8.72820020e-02,
         1.72331437e-01, -1.40289262e-01, -4.68482897e-02,
         3.97892036e-02,  2.47199297e-01,  1.05222411e-

In [57]:
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

# 문서를 토큰화하여 Word2Vec 모델에 학습시키기 위한 형태로 변환
tokenized_documents = [simple_preprocess(document, min_len=1, max_len=100) for document in processed_documents]

# Word2Vec 모델 학습 (Skip-gram)
word2vec_model = Word2Vec(sentences=tokenized_documents, vector_size=100, window=5, min_count=1, workers=4, sg=1)

# 각 문서에 대한 임베딩 추출
embedded_documents_word2vec = [word2vec_model.wv[document] for document in tokenized_documents]

# 임베딩된 결과 출력
print(embedded_documents_word2vec[:1])


[array([[ 1.04013346e-01,  3.82757753e-01, -1.44969672e-01,
         1.22362264e-01, -8.14769864e-02, -3.88898611e-01,
         7.49154016e-02,  4.93013263e-01,  7.03305937e-03,
        -7.98221231e-02, -1.55496281e-02, -1.98066160e-01,
        -1.46349579e-01,  1.30856752e-01,  1.43828960e-02,
         3.55697274e-02,  2.43729278e-01, -2.39828587e-01,
        -1.34869277e-01, -5.98113060e-01,  3.26761216e-01,
         6.17512241e-02,  2.40892485e-01, -1.49849340e-01,
         3.65146585e-02,  7.35051483e-02, -1.92503110e-01,
        -1.11215882e-01, -2.14650854e-01,  1.13203131e-01,
         2.68151551e-01, -1.38717726e-01,  2.60504872e-01,
        -3.72954786e-01, -1.01905480e-01,  1.06117308e-01,
         4.86576855e-02,  2.01111399e-02, -1.71417281e-01,
        -3.71253550e-01, -1.03388252e-02, -2.88842320e-01,
         4.39958349e-02, -3.42183337e-02,  1.15106121e-01,
        -2.77173936e-01, -3.76055330e-01, -5.56928366e-02,
         8.32285509e-02,  2.78110802e-01, -5.62827177e-